### 워크플로우 이니시

In [1]:
# 특정 워크플로우의 초기는 다음과 같이 생긴다

from entity.process import GuardCondition, TaskSpec, TaskType, Layer, AgentNature, AgentRole, Process
from entity.validators import TokenValidator, SpecChainValidator
from entity.tokens import Token
from core.utils import load_resource_specs
import os
import logging
from core import logging_utils, utils

log_levels = {
    "DEBUG": logging.DEBUG,
    "INFO": logging.INFO,
    "WARNING": logging.WARNING,
    "ERROR": logging.ERROR,
    "CRITICAL": logging.CRITICAL
}

log_level = log_levels.get(os.environ.get("LOG_LEVEL", "INFO").upper(), logging.INFO)

logging_utils.setup_logging(
    log_dir="logs",
    log_level=log_level,
    max_bytes=10 * 1024 * 1024, 
    backup_count=5, 
    console_output=True 
)

logging_utils.cleanup_old_logs(log_dir="logs", days_to_keep=7)


16:02:33 - core.logging_utils - INFO - Logging initialized - logs directory: c:\Users\kakao\Desktop\AI_Agent\Semantic Layer\logs
16:02:33 - core.logging_utils - INFO - Log rotation configured: max 10.0MB per file, keeping 5 backups


In [2]:
resource_db = load_resource_specs("./ResourceSpec/TokenSpec.yaml")
validator = TokenValidator(resource_db)

# Chain Validator 인스턴스 생성
chain_check = SpecChainValidator(validator)

# 최초 토큰 생성
valid_token = Token(
    trace_id = "dfhou3898dfalss28fhs", # 향후 랜덤 생성기 필요
    content = {
        "text" : "Samsung Elec Risk Analysis",
        "risk_score" : 0.96
    }
)

### 연관 태스크 정의
## Task는 Transition이다 (화살표다)
task_A = TaskSpec(
    # 일반 설명
    task_id = "TASK_TEST_001",
    description="금융 뉴스 감성 분석",
    type=TaskType.PYTHON_FUNC, 
    target="utils.analysis:calculate_sentiment", # 실행 함수 경로

    # config
    config = {
        'business_context' : 'ojs가 분석을 위해 만든 최초 example'
    },

    # 구조
    layer = Layer.OBSERVATION,
    required_agent_roles=[AgentRole.CONSULTANT],
    required_agent_types=[AgentNature.LLM],

    # 가드 구조 선언
    guards = [
        GuardCondition(
            target_topic_id = "TOPIC_FINANCE", 
            min_relevance=0.7,
            description="금융 관련성 0.7 이상 필수"
        )
    ],

    # TokenSpec.yaml 파일 참고
    input_spec_id="RS_RISK_TOKEN_V1",
    output_spec_id="RS_RISK_TOKEN_V2"
)

### 연관 태스크 정의
## Task는 Transition이다 (화살표다)
task_B = TaskSpec(
    # 일반 설명
    task_id = "TASK_TEST_002",
    description="분석 결과 추가 분석",
    type=TaskType.PYTHON_FUNC,  #
    target="utils.analysis:calculate_sentiment", # 실행 함수 경로 (LLM INVOKE 경로. role/type도 함께 전달)

    # config
    config = {
        'business_context' : 'ojs가 분석을 위해 만든 최초 example'
    },

    # 구조
    layer = Layer.OBSERVATION,
    required_agent_roles=[AgentRole.CONSULTANT],
    required_agent_types=[AgentNature.LLM],

    # 가드 구조 선언
    guards = [
        GuardCondition(
            target_topic_id = "TOPIC_FINANCE", 
            min_relevance=0.7,
            description="금융 관련성 0.7 이상 필수"
        )
    ],

    # TokenSpec.yaml 파일 참고
    input_spec_id="RS_RISK_TOKEN_V2",
    output_spec_id="RS_RISK_TOKEN_V2",

)


In [3]:
task_C = TaskSpec(
    # 일반 설명
    task_id = "TASK_TEST_003",
    description="분석 결과 추가 분석",
    type=TaskType.PYTHON_FUNC,  #
    target="utils.analysis:calculate_sentiment", # 실행 함수 경로 (LLM INVOKE 경로. role/type도 함께 전달)

    # config
    config = {
        'business_context' : 'ojs가 분석을 위해 만든 최초 example'
    },

    # 구조
    layer = Layer.OBSERVATION,
    required_agent_roles=[AgentRole.CONSULTANT],
    required_agent_types=[AgentNature.LLM],

    # 가드 구조 선언
    guards = [
        GuardCondition(
            target_topic_id = "TOPIC_FINANCE", 
            min_relevance=0.7,
            description="금융 관련성 0.7 이상 필수"
        )
    ],

    # TokenSpec.yaml 파일 참고
    input_spec_id="RS_RISK_TOKEN_V2",
    output_spec_id="RS_RISK_TOKEN_V2"

)

task_D = TaskSpec(
    # 일반 설명
    task_id = "TASK_TEST_004",
    description="분석 결과 추가 분석",
    type=TaskType.PYTHON_FUNC,  #
    target="utils.analysis:calculate_sentiment", # 실행 함수 경로 (LLM INVOKE 경로. role/type도 함께 전달)

    # config
    config = {
        'business_context' : 'ojs가 분석을 위해 만든 최초 example'
    },

    # 구조
    layer = Layer.OBSERVATION,
    required_agent_roles=[AgentRole.CONSULTANT],
    required_agent_types=[AgentNature.LLM],

    # 가드 구조 선언
    guards = [
        GuardCondition(
            target_topic_id = "TOPIC_FINANCE", 
            min_relevance=0.7,
            description="금융 관련성 0.7 이상 필수"
        )
    ],

    # TokenSpec.yaml 파일 참고
    input_spec_id="RS_RISK_TOKEN_V2",
    output_spec_id="RS_RISK_TOKEN_V2"
)


### 프로세스 테스트

In [4]:
# 최초 선언시 아이디 선언
process_test = Process("ojs_test_process")

# (ORANGE) 나중에 Process 선언문 읽고 아래 action이 자동 수행되는 로직이 필요
process_test.tasks
process_test.add_task(task_A)
process_test.add_task(task_B)
process_test.add_link(task_A, task_B)

# 그래프 구조 완성된거 확인 가능
process_test.graph
process_test.compile(chain_validator=chain_check)

16:02:49 - ChainValidator - INFO - [LINK] TOKEN SPEC 일치 TASK_TEST_001 -> TASK_TEST_002
16:02:49 - Proc_ojs_test_process - INFO - [COMPILE SUCCESS] Spec 일치 & Process Topology 정상.


True

##### ANDOR Join Test

In [5]:
proc = Process("RISK_MODEL_FLOW")

# 2. 태스크 등록
proc.add_task(task_A) # 재무
proc.add_task(task_B) # 거시
proc.add_task(task_C) # 통합 모델
proc.add_task(task_D) # 발표 모델

In [6]:
proc.add_link(task_A, task_C) # 재무 -> 통합
proc.add_link(task_B, task_C) # 거시 -> 통합
proc.add_link(task_C, task_D)

In [7]:
proc.compile(chain_check)

16:02:56 - ChainValidator - INFO - [LINK] TOKEN SPEC 일치 TASK_TEST_001 -> TASK_TEST_003
16:02:56 - ChainValidator - INFO - [LINK] TOKEN SPEC 일치 TASK_TEST_002 -> TASK_TEST_003
16:02:56 - ChainValidator - INFO - [LINK] TOKEN SPEC 일치 TASK_TEST_003 -> TASK_TEST_004
16:02:56 - Proc_RISK_MODEL_FLOW - INFO - [COMPILE SUCCESS] Spec 일치 & Process Topology 정상.


True

In [9]:
proc.inject_token("TASK_TEST_001", valid_token)

16:04:07 - Proc_RISK_MODEL_FLOW - INFO - 토큰 주입 = TASK_TEST_001


In [10]:
proc.token_queue

deque([('TASK_TEST_001',
        
        [Token: dfhou3898dfalss28fhs]
         ├─ Timestamp: 2026-01-30 16:02:37.630
         └─ Content  : Samsung Elec Risk Analysis...)])

### /engine/engine.py

In [1]:
import importlib
import logging
from dataclasses import dataclass, field
from typing import Any, Optional, Dict, List
from datetime import datetime

In [ ]:
# [Dependency] 앞서 정의한 FiringResult
@dataclass
class FiringResult:
    task_id: str
    success: bool
    message: str
    new_token: Optional[Any] = None
    elapsed_ms: float = 0.0
    routes_triggered: int = 0

class ExecEngine:
    """
    핵심 프로세스 실행 엔진
    CSPN의 Transition(Task)을 실행하고, 토큰을 변환/생성하는 역할 수행
    
    내부 절차 : 가드 체크 -> 입력 확인 -> 함수 실행 -> 아웃풋 확인 -> 토큰 갱신 -> 라우팅
    """
    def __init__(self, token_validator):
        self.tv = token_validator
        self.logger = logging_utils.get_logger(f"Engine")

    def run_step(self, process: Process) -> Optional[FiringResult]:
        """
        한 스텝의 태스크를 꺼내 처리함
        """
        # 1. 프로세스 큐에서 Job를 추출
        if not process.token_queue:
            self.logger.warning(f"[RUN] 처리할 토큰이 큐에 없습니다.")
            return None
        
        # 2. 큐에서 토큰과 태스크 아이디 추출
        target_task_id, token = process.token_queue.popleft()
        task = process.tasks[target_task_id]

        start_time = datetime.now()

        # 2. 태스크 실행 전 가드 체크
        if not self._check_guards(task, token):
            return FiringResult(task.task_id, False, message="Guard Condition Failed")
        
        # 3. Input Spec Validation
        try:
            self.tv.validate(token.content, task.input_spec_id)
        except Exception as e:
            # SemanticError, ValueError(Spec 없음) 등 모든 검증 에러 포착   
            self.logger.warning(f"Task {task.task_id} Input Validation Failed: {e}")
            return FiringResult(task.task_id, False, f"Input Spec Fail: {str(e)}")
        
        # 4. 동적 실행
        # task.target은 해당 태스크에 할당된 function 혹은 API 실행을 의미
        try:
            func = self._resolve_function(task.target)
            
            # 실행 : Token Content + task config 주입
            output_content = func(token.content, **task.config)

        except Exception as e:
            self.logger.error(f"Task {task.task_id} Execution Logic Failed: {e}", exc_info=True)
            return FiringResult(task.task_id, False, f"Runtime Execution Error: {str(e)}")

        # 5. [Validator] Output Spec Validation
        try:
            self.tv.validate(output_content, task.output_spec_id)
        except Exception as e:
            self.logger.error(f"Task {task.task_id} Output Validation Failed: {e}", exc_info=True)
            return FiringResult(task.task_id, False, f"Output Spec Fail: {str(e)}")

        # 6. Token Evolution (State Update)
        new_token = self._evolve_token(token, output_content, task)

        # 7. Routing (PetriNet Propagation)
        # Process에게 토큰 도착 알림 (Process 내부 로직으로 병합/대기 수행)
        routes_count = self._propagate_token(process, task, new_token)

        elapsed = (datetime.now()-  start_time).total_seconds() * 1000
        return FiringResult(task.task_id, True, "Success", new_token, elapsed, routes_count)

    ### ===== 내부 로직 ===== ###

    def _check_guards(self, task: Any, token: Any) -> bool:
        """Topic 가중치 기반 실행 조건 평가"""
        if not task.guards:
            return True
        
        token_topics = getattr(token, 'topics', {})
        for guard in task.guards:
            score = token_topics.get(guard.target_topic_id, 0.0)
            if score < guard.min_relevance:
                self.logger.debug(f"Guard Fail: {guard.target_topic_id} ({score} < {guard.min_relevance})")
                return False
        return True

    def _resolve_function(self, target_path: str):
        """'module.path:func_name' 문자열을 실제 함수 객체로 변환"""
        try:
            module_path, func_name = target_path.split(":")
            module = importlib.import_module(module_path)
            return getattr(module, func_name)
        except (ValueError, ImportError, AttributeError) as e:
            raise ImportError(f"Function resolution failed for '{target_path}': {e}")

    def _evolve_token(self, old_token: Any, new_content: Dict, task: Any) -> Any:
        """이전 토큰 승계 및 이력 업데이트"""
        # Token 클래스 정의에 따라 model_copy 또는 생성자 사용
        # 예시: Pydantic model_copy
        new_history = getattr(old_token, 'history', []) + [task.task_id]
        
        return old_token.model_copy(update={
            "content": new_content,
            "history": new_history
        })

    def _propagate_token(self, process: Any, current_task: Any, token: Any) -> int:
        """
        [Routing Logic]
        다음 Task들을 찾아 'Process.arrive_token'을 호출함.
        """
        next_tasks = process.get_next_nodes(current_task.task_id)
        
        if not next_tasks:
            process.completed_tokens.append(token) # End of Chain
            return 0

        triggered = 0
        for next_task in next_tasks:
            # Look-ahead Guard Check: 갈 자격이 있는 경로인가?
            if self._check_guards(next_task, token):
                # [핵심] Process의 Place 로직 호출 (Petri Net 동기화 위임)
                process.arrive_token(
                    from_task_id=current_task.task_id,
                    to_task_id=next_task.task_id,
                    token=token
                )
                triggered += 1
            else:
                self.logger.info(f"Route Ignored: {current_task.task_id} -> {next_task.task_id}")
        
        return triggered
